<a href="https://colab.research.google.com/github/DenisShevchenkoV/tyumile/blob/master/%D0%93%D0%B5%D0%BD%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D1%8F_%D1%81%D0%B0%D0%B9%D1%82%D0%BE%D0%B2_%D0%B8%D0%B7_%D0%BF%D0%B5%D1%80%D0%B5%D0%B2%D0%BE%D0%B4%D0%BE%D0%B2_1_0%2C_%D0%A6%D0%9F%D0%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U easynmt
!pip install python-wordpress-xmlrpc
!pip install wget
output.clear()

In [ ]:
import pandas as pd
import numpy as np
import os #Для работы с файлами 
import time
from google.colab import drive
import traceback
import xml.etree.ElementTree as ET
from xml.etree import ElementTree
import xlrd
import zipfile
import re
from lxml import html, etree
import requests
import httplib2
import json
import wget
import datetime as DT
from datetime import date
from easynmt import EasyNMT

# Обработка записей и загрузка данных в WP
from wordpress_xmlrpc import Client, WordPressPost
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import media
from wordpress_xmlrpc.compat import xmlrpc_client

drive.mount('/content/drive') #rusexpress.online
base_path = '/content/drive/MyDrive/generate_sites/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Функции для arxiv.org

In [ ]:
!unzip -q /content/drive/MyDrive/generate_sites/arxiv-metadata-oai-snapshot.json.zip


In [ ]:
arxiv_json = []
for line in open('/content/arxiv-metadata-oai-snapshot.json', 'r'):
    arxiv_json.append(json.loads(line))

In [ ]:
def filterByDate(date, arxiv_json): #поиск нужной даты в списке
  filtered_by_date = []
  for article in arxiv_json:
    if date in article['update_date']:
      filtered_by_date.append(article)
  return filtered_by_date

def filterByTheme(theme, arxiv_json): #поиск нужной даты в списке
  filtered_by_theme = []
  for article in arxiv_json:
    if theme in article['categories']:
      filtered_by_theme.append(article)
  return filtered_by_theme

def cleanText(text):
  badwords = ['$', '\n']
  for word in badwords:
    text = text.replace(word,' ')
  return text

def getPageHtml(url):
  page = {}
  r = requests.get(url)
  html_page = r.text
  return html_page

def getPagesUrls():
  pass

def getArticlesUrls():
  pass

# переводы
def translateText(text):
  pass

In [ ]:
date = '2021-04-16'
filtered_by_date = filterByDate(date, arxiv_json)
len(filtered_by_date)

1123

In [ ]:
text = filtered_by_date[1]['abstract']
text = cleanText(text)
del arxiv_json
del line

In [ ]:
from easynmt import EasyNMT
model = EasyNMT('m2m_100_1.2B')

100%|██████████| 89.9k/89.9k [00:00<00:00, 424kB/s]


# Перевод статей


In [ ]:
text = filtered_by_date[10]['abstract']
text = cleanText(text)
text

"  For an exotic locally compact Hausdorff space  L , constructed under the assumption of the Ostaszewski's  \\clubsuit -principle, and a countable ordinal space  \\alpha , we prove that all operators defined on  C_0(\\alpha\\times L)  are as simple as possible. We also investigate the geometry of such space  C_0(\\alpha\\times L)  and we classify up to isomorphisms all its complemented subspaces. "

In [ ]:
import tqdm
text = '<p>Just because you’re disconnecting from the news, doesn’t mean you have to disconnect from your community. In fact, staying in touch with <a href="https://newpaltz.edu/media/idmh/covid-19/IDMH%20COVID19%20Community%20Stress%20Management%20Tip%20Sheet%202020%20Final.pdf" target="_blank" rel="noopener" data-outbound-instanced="true">friends and family, online or in person</a> can help relieve stress. If you’re not likely to get embroiled in a tense political conversation, staying in touch with your social networks—and staying in touch with yourself—does a lot to ease news exhaustion.</p>'
translated_text = model.translate(text, target_lang='ru',  beam_size=20)
translated_text

'Просто потому, что вы отключаетесь от новостей, это не означает, что вам нужно отключиться от вашего сообщества. На самом деле, оставаясь в контакте с <a href="https://newpaltz.edu/media/idmh/covid-19/IDMH%20COVID19%20Community%20Stress%20Management%20Tip%20Sheet%202020%20Final.pdf" target="_blank" rel="noopener" data-outbound-instanced="true">друзья и семья, онлайн или лично</a> может помочь облегчить стресс. Если вы вряд ли попадете в напряженный политический разговор, оставаться в контакте с вашими социальными сетями - и оставаться в контакте с самим собой - делает многое, чтобы облегчить истощение новостей.</p>'

# Перевод всего архива

In [ ]:
theme = 'cs.AI'
filtered_by_theme = filterByTheme(theme, arxiv_json)
len(filtered_by_theme)

32541

In [ ]:
del line
del arxiv_json

In [ ]:
# создание модели для перевода
from easynmt import EasyNMT
model = EasyNMT('m2m_100_418M')

In [ ]:
# перевод всех статей по очереди
import tqdm
cur_time = time.time()
for i in range(len(filtered_by_theme)):
  text = filtered_by_theme[i]['abstract']
  title = filtered_by_theme[i]['title']

  text = cleanText(text)
  title = cleanText(title)

  translated_text = model.translate(text, target_lang='ru',  beam_size=20)
  translated_title = model.translate(title, target_lang='ru',  beam_size=20)

  filtered_by_theme[i]['abstract'] = translated_text
  filtered_by_theme[i]['title'] = translated_title
  
  print('Закончили перевод статьи {}, длина текста {}, прошло времени {}'.format(i, len(text), time.time()-cur_time))
  if i%50 == 0:
    with open(base_path+'filtered_by_theme.json', 'w') as outfile:
      json.dump(filtered_by_theme, outfile)

Закончили перевод статьи 0, длина текста 1175, прошло времени 26.102351188659668
Закончили перевод статьи 1, длина текста 953, прошло времени 34.591448068618774
Закончили перевод статьи 2, длина текста 793, прошло времени 39.084234952926636
Закончили перевод статьи 3, длина текста 826, прошло времени 44.372050285339355
Закончили перевод статьи 4, длина текста 943, прошло времени 50.63851761817932
Закончили перевод статьи 5, длина текста 227, прошло времени 53.549694299697876
Закончили перевод статьи 6, длина текста 266, прошло времени 56.03613638877869
Закончили перевод статьи 7, длина текста 1188, прошло времени 65.6506073474884
Закончили перевод статьи 8, длина текста 1348, прошло времени 74.06339454650879
Закончили перевод статьи 9, длина текста 1032, прошло времени 85.8649537563324
Закончили перевод статьи 10, длина текста 1601, прошло времени 96.4291741847992
Закончили перевод статьи 11, длина текста 566, прошло времени 99.904714345932
Закончили перевод статьи 12, длина текста 480

Exception: CUDA out of memory. Tried to allocate 118.00 MiB (GPU 0; 14.76 GiB total capacity; 11.17 GiB already allocated; 35.75 MiB free; 13.69 GiB reserved in total by PyTorch)


RuntimeError: ignored

In [ ]:
filtered_by_theme = []
for line in open(base_path+'/filtered_by_theme.json', 'r'):
    filtered_by_theme.append(json.loads(line))
filtered_by_theme[0][105]

{'abstract': '  В этой статье описывается система, способная полуавтоматически заполнять шаблон XML из бесплатных текстов в клинической области (практические рекомендации). Шаблон XML содержит семантическую информацию, не зашифрованную в тексте (пары условий и действий/рекомендаций). Поэтому необходимо рассчитать точный диапазон условий над текстовыми последовательностями, выражающими необходимые действия. В этой статье мы представим правила, разработанные для этой задачи. Мы показываем, что система дает хорошие результаты при применении к анализу принципов французской практики. ',
 'authors': 'Amanda Bouffier (LIPN)',
 'authors_parsed': [['Bouffier', 'Amanda', '', 'LIPN']],
 'categories': 'cs.AI',
 'comments': None,
 'doi': '10.1007/978-3-540-76298-0_69',
 'id': '0709.4015',
 'journal-ref': "Dans International Semantic Web Conference Proceedings -\n  International Semantic Web Conference-Doctoral Consortium, Busan : Cor\\'ee,\n  R\\'epublique de (2007)",
 'license': None,
 'report-no'

# Парсинг сайта с переводом

In [ ]:
url = 'https://www.yogajournal.com/lifestyle/balance/stress-balance/how-to-manage-news-trauma/'
html_page = getPageHtml(url)
html_page

'<!DOCTYPE html>\n<html class=>\n  <head>\n  <meta charset="UTF-8" />\n  <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n  <meta http-equiv="X-UA-Compatible" content="IE=edge"><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={licenseKey:"NRJS-48b9ed1c0b517b29335",applicationID:"882436314"};window.NREUM||(NREUM={}),__nr_require=function(e,t,n){function r(n){if(!t[n]){var i=t[n]={exports:{}};e[n][0].call(i.exports,function(t){var i=e[n][1][t];return r(i||t)},i,i.exports)}return t[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(e,t,n){function r(){}function i(e,t,n){return function(){return o(e,[u.now()].concat(c(arguments)),t?null:this,n),t?void 0:this}}var o=e("handle"),a=e(7),c=e(8),f=e("ee").get("tracer"),u=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var d=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","i

In [ ]:
text = html_page.split('<p>')[1:4]
text = ' '.join(map(str, text))
text

'Whether it’s yet another video of a police altercation gone wrong or alerts about the latest mass shooting, we face a seemingly unrelenting deluge of disturbing news in our media feeds, timelines, and DMs on a daily basis. And it&#8217;s wearing us down.</p>\n About two-thirds of Americans say they&#8217;re worn out by the amount of news that&#8217;s coming at them, according to a<a href="https://www.pewresearch.org/fact-tank/2020/02/26/almost-seven-in-ten-americans-have-news-fatigue-more-among-republicans/" target="_blank" rel="noopener"> Pew Research Center study</a>. The 24-hour news cycle is not just exhausting; it can be traumatic—so much so that researchers believe <a href="https://www.dailymail.co.uk/health/article-3071103/Online-videos-cause-post-traumatic-stress-disorder.html#ixzz4hwcU8bKf" target="_blank" rel="noopener">watching violent videos</a> causes symptoms of post-traumatic stress disorder (PTSD). Even people who view videos of merely confrontational situations suffer

In [ ]:
translated_text = model.translate(text, target_lang='ru',  beam_size=20)
translated_text

'Будь то еще одно видео полицейского столкновения, которое пошло не так, или предупреждения о последней массовой стрельбе, мы ежедневно сталкиваемся с, казалось бы, неустанным потоком тревожных новостей в наших средствах массовой информации, расписаниях и СМИ. И это&#8217;s носить нас вниз.</p>\n Около двух третей американцев говорят, что они исчерпаны количеством новостей, которые приходят к ним, согласно <a href="https://www.pewresearch.org/fact-tank/2020/02/26/almost-seven-in-ten-Americans-have-news-fatigue-more-among-republicans/" target="_blank" rel="noopener"> исследование Pew Research Center</a>. 24-часовой цикл новостей не просто истощает; он может быть травматичным — настолько, что исследователи считают <a href="https://www.dailymail.co.uk/health/article-3071103/Online-videos-cause-post-traumatic-stress-disorder.html#ixzz4hwcU8bKf" target="_blank" rel="noopener">просмотр насильственных видео</a> вызывает симптомы посттравматического стрессового расстройства (ПТСР). Даже люди, 

# Добавление данных в WP

## Через REST API

In [ ]:
import base64
from datetime import datetime

base_url = 'https://tyumile-joomla.myjino.ru/wp-json/wp/v2'
user = 'tyumile'
password = 'dKua 6Wuq LxsT 5C5i 7vho 6gRL'
creds = user + ':' + password
token = base64.b64encode(creds.encode())
header = {'Authorization':'Basic' + token.decode('utf-8')}
post = {
    # 'date': "2021-11-08 03:01:46",
    # 'id':'9',
    'title':'Newnewnew 4444',
    'content':'My first API post contentf f sdf s ',
    # 'tags':['AI', 'DS'],
    # 'status': 'publish',
    # 'User-Agent':'test',
    'author': '1'
}


r = requests.post(base_url + '/posts', headers = header, json = post, verify=False)
print(r.text)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[{"id":11,"date":"2021-04-29T13:50:26","date_gmt":"2021-04-29T10:50:26","guid":{"rendered":"http:\/\/tyumile-joomla.myjino.ru\/?p=11"},"modified":"2021-04-29T13:50:26","modified_gmt":"2021-04-29T10:50:26","slug":"53w45w3","status":"publish","type":"post","link":"http:\/\/tyumile-joomla.myjino.ru\/2021\/04\/29\/53w45w3\/","title":{"rendered":"53w45w3"},"content":{"rendered":"\n<p>45sgsdfgsg<\/p>\n","protected":false},"excerpt":{"rendered":"<p>45sgsdfgsg<\/p>\n","protected":false},"author":1,"featured_media":0,"comment_status":"open","ping_status":"open","sticky":false,"template":"","format":"standard","meta":[],"categories":[1],"tags":[],"_links":{"self":[{"href":"http:\/\/tyumile-joomla.myjino.ru\/wp-json\/wp\/v2\/posts\/11"}],"collection":[{"href":"http:\/\/tyumile-joomla.myjino.ru\/wp-json\/wp\/v2\/posts"}],"about":[{"href":"http:\/\/tyumile-joomla.myjino.ru\/wp-json\/wp\/v2\/types\/post"}],"author":[{"embeddable":true,"href":"http:\/\/tyumile-joomla.myjino.ru\/wp-json\/wp\/v2\/users

In [ ]:
# JWC

import base64
from datetime import datetime

base_url = 'http://tyumile-joomla.myjino.ru//wp-json/jwt-auth/v1/token'
user = 'tyumile'
password = 'secretkey'
creds = user + ':' + password
token = base64.b64encode(creds.encode())
header = {'Authorization':'Basic' + token.decode('utf-8')}
# header = {'username':'tyumile','password':'12131415'}
post = {
    # 'date': "2021-11-08 03:01:46",
    # 'id':'9',
    'title':'Changed hd ',
    'content':'My first API post contentf f sdf s ',
    # 'tags':['AI', 'DS'],
    # 'status': 'publish',
    # 'User-Agent':'test',
    'author': '1'
}


r = requests.post(base_url, headers = header, json = post)
print(r)

In [ ]:
r.text

'[{"id":11,"date":"2021-04-29T13:50:26","date_gmt":"2021-04-29T10:50:26","guid":{"rendered":"http:\\/\\/tyumile-joomla.myjino.ru\\/?p=11"},"modified":"2021-04-29T13:50:26","modified_gmt":"2021-04-29T10:50:26","slug":"53w45w3","status":"publish","type":"post","link":"http:\\/\\/tyumile-joomla.myjino.ru\\/2021\\/04\\/29\\/53w45w3\\/","title":{"rendered":"53w45w3"},"content":{"rendered":"\\n<p>45sgsdfgsg<\\/p>\\n","protected":false},"excerpt":{"rendered":"<p>45sgsdfgsg<\\/p>\\n","protected":false},"author":1,"featured_media":0,"comment_status":"open","ping_status":"open","sticky":false,"template":"","format":"standard","meta":[],"categories":[1],"tags":[],"_links":{"self":[{"href":"http:\\/\\/tyumile-joomla.myjino.ru\\/wp-json\\/wp\\/v2\\/posts\\/11"}],"collection":[{"href":"http:\\/\\/tyumile-joomla.myjino.ru\\/wp-json\\/wp\\/v2\\/posts"}],"about":[{"href":"http:\\/\\/tyumile-joomla.myjino.ru\\/wp-json\\/wp\\/v2\\/types\\/post"}],"author":[{"embeddable":true,"href":"http:\\/\\/tyumile-jo

In [ ]:
# import requests
# from requests.adapters import HTTPAdapter
# from requests.packages.urllib3.util.retry import Retry

# session = requests.Session()
# retry = Retry(connect=3, backoff_factor=0.5)
# adapter = HTTPAdapter(max_retries=retry)
# session.mount('http://', adapter)
# session.mount('https://', adapter)

# session.get(base_url + '/posts')

In [ ]:
response = requests.get(base_url+'/posts', verify=False)
response.status_code

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


200

## XMLRPC

In [ ]:
def initWpPost():
  wp = Client('http://tyumile-joomla.myjino.ru/xmlrpc.php', 'tyumile', '12131415')
  post = WordPressPost()
  return wp, post

def uploadMedia(parsed_media): #media - лист ссылок на картинки и видео
  wp_media_links = []  # ссылки на медиа в ВП для статей
  for i in range(len(parsed_media)):
    media_link = parsed_media[i]['link']
    media_type = parsed_media[i]['type']
    data = {
            'name': media_link.split('/')[-1],
            'type': media_type  #'image/jpeg',  # mimetype
    }

    # загрузка изображения и отправка в WP
    h = httplib2.Http('.cache')
    response, content = h.request(media_link)

    data['bits'] = content
    response = client.call(media.UploadFile(data))
    wp_media_links.append(response['link'])

  return wp_media_links 

def getWpPostMetadata(post, parsed_post, post_status='publish'):
  post.title = parsed_post['title']
  post.content = parsed_post['content']
  post.post_status = post_status
  post.terms_names = {
    'post_tag': post_data['tags'],
    'category': post_data['categories']
  }

def addNewPost(parsed_data, parsed_media, wp):

  wp_media_links = uploadMedia(parsed_media) # получаем список ссылок для замены на наши
  post = getWpPostMetadata(parsed_post) # формируем данные для поста

  post_id = wp.call(NewPost(post))
  print('Добавлена статья ', post_id)


In [ ]:
wp, post = initWpPost()
addNewPost(parsed_data, parsed_media, wp)

# Сборка

## ParserTranslator

In [ ]:
!unzip -q /content/drive/MyDrive/generate_sites/arhives/arxiv-metadata-oai-snapshot.json.zip

In [ ]:
import subprocess
from subprocess import call

class ParserTranslator(): # сбор данных с сайтов и создание единого JSON с данными

  def __init__(self):

    self.base_path = '/content/drive/MyDrive/generate_sites/'
    self.dir_arhives = 'arhives/'
    self.date = '2021-18-02' #date.today()

    self.beams = 20
    # self.model = EasyNMT('m2m_100_418M') # модель для переводов

# скачиваем архивы
    # Arxiv.org
    # urlArxiv = base_path+self.dir_arhives+'arxiv-metadata-oai-snapshot.json.zip'
    # wget.download(url, 'arxiv_org_json.zip')
    # call('!unzip -q /content/drive/MyDrive/generate_sites/arhives/arxiv-metadata-oai-snapshot.json.zip') # + self.base_path + self.dir_arhives + 'arxiv-metadata-oai-snapshot.json.zip') # распаковываем свеже скачанный архив json
    self.arxiv_json = []
    for line in open('/content/arxiv-metadata-oai-snapshot.json', 'r'):
      self.arxiv_json.append(json.loads(line))

    # открываем полную базу статей
    self.fullBaseJson = []
    for line in open(self.base_path + 'fullBaseJson.json', 'r'):
        self.fullBaseJson.append(json.loads(line))


#############################

  def fullBaseUpdate(self, json_file):
    for article in json_file:
      final_article = {
          'title': article          
      }
      self.fullBaseJson.append(article)
    return self.fullBaseJson

#############################
  def parseArxiv(self, method = 'update'):

    def getFilteredArxiv(self, method):
      cur_time = time.time()
      print('Начали фильтровать архив по дате {}, метод {}'.format(self.date, method))

      filtered_arxiv_json = []
      for article in self.arxiv_json:
        print(article['update_date'])
        # if method == 'update':
        #   if self.date in article['update_date']:
        #     filtered_arxiv_json.append(article)
        # else:
        #   filtered_arxiv_json.append(article)

      print('Завершили фильтровтаь записи')
      print(filtered_arxiv_json[0])

      return filtered_arxiv_json

    def translateArxiv(self, filtered_arxiv_json):

      for i in range(0, 10): #len(filtered_arxiv_json)):
        text = filtered_arxiv_json[i]['abstract']
        title = filtered_arxiv_json[i]['title']

        text = cleanText(text)
        title = cleanText(title)

        translated_text = self.model.translate(text, target_lang='ru',  beam_size=self.beams)
        translated_title = self.model.translate(title, target_lang='ru',  beam_size=self.beams)

        filtered_arxiv_json[i]['abstract'] = translated_text
        filtered_arxiv_json[i]['title'] = translated_title

      return filtered_arxiv_json

    filtered_arxiv_json = getFilteredArxiv(self, method)
    translated_arxiv = translateArxiv(self, filtered_arxiv_json)
    self.fullBaseJson = fullBaseUpdate(translated_arxiv)

    return self.fullBaseJson

############################

  def parseGithub():
    pass

  def saveFullBaseToJson(self):  # сохраняет сформированный json в файл, при необходимости разбивает
    with open(self.base_path + 'fullBaseJson.json', 'w') as outfile:
      json.dump(self.fullBaseJson, outfile)
    print('Сохранили fullBaseJson.json')

############################

  def parseAllData(self, method = 'update'):

    fullBaseJson = self.parseArxiv(self.fullBaseJson, method)
    # fullBaseJson = parseGithub(fullBaseJson)

    saveFullBaseToJson(fullBaseJson)

    return fullBaseJson

In [ ]:
# filtered_arxiv_json = []
# for article in open('arxiv-metadata-oai-snapshot.json', 'r'):
#   print(article['update_date'])

## SiteUpdater

In [ ]:
class SiteUpdater(object): # добавление данных на сайты и обновление записей

  def __init__(self):
    self.sites_vars = {
      'site_ds':{
          'url':'http://tyumile-joomla.myjino.ru/xmlrpc.php',
          'user':'tyumile',
          'password:':'12131415',
          'arxiv_categories':['AI'],
          'medium_categories':['AI'],
          'towards_categories':['AI'],
          'site_categories':{
              'arxiv':'Исследования',
              'github':'Модули'
          }

      },
      'site_medicine':{
          'user':'tyumile',
          'password:':'12131415',
          'url':''
      },
    }

  def initWpPost(self, site):

    site_vars = self.site_vars(site)
    wp = Client(site_vars['url'], site_vars['user'], site_vars['password'])
    post = WordPressPost()

    return wp, post

  def uploadMedia(self, parsed_media): #media - лист ссылок на картинки и видео
    wp_media_links = []  # ссылки на медиа в ВП для статей
    for i in range(len(parsed_media)):
      media_link = parsed_media[i]['link']
      media_type = parsed_media[i]['type']
      data = {
              'name': media_link.split('/')[-1],
              'type': media_type  #'image/jpeg',  # mimetype
      }

      # загрузка изображения и отправка в WP
      h = httplib2.Http('.cache')
      response, content = h.request(media_link)

      data['bits'] = content
      response = client.call(media.UploadFile(data))
      wp_media_links.append(response['link'])

    return wp_media_links 

  def getWpPostMetadata(self, post, parsed_post, post_status='publish'):
    post.title = parsed_post['title']
    post.content = parsed_post['content']
    post.post_status = post_status
    post.terms_names = {
      'post_tag': post_data['tags'],
      'category': post_data['categories']
    }

  def addArticles(self, site):

    wp, post = initWpPost(site)


    # wp_media_links = uploadMedia(parsed_media) # получаем список ссылок для замены на наши
    # post = getWpPostMetadata(parsed_post) # формируем данные для поста

    # post_id = wp.call(NewPost(post))
    # print('Добавлена статья ', post_id)

# Запуск

In [ ]:
parser = ParserTranslator()

In [ ]:
# fullBaseJson = parser.parseAllData(method='full')

In [ ]:
method = 'update'
filtered_arxiv_json = parser.parseArxiv(method)

In [ ]:
/content/drive/MyDrive/generate_sites/arhives/arxiv-metadata-oai-snapshot.json.zip

In [ ]:
parser = ParserTranslator()
fullBaseJson = parser.parseAllData()

siteUpdater = SiteUpdater(fullBaseJson)

site_ds_logs = siteUpdater.addArticles(site='site_ds')
site_medicine_logs = siteUpdater.addArticles(site='site_medicine')